# Imports

In [13]:
import pandas as pd

from sklearn.model_selection import train_test_split
import tensorflow as tf

2025-02-27 10:57:17.498892: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-27 10:57:17.740552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740664637.846611   18872 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740664637.875632   18872 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 10:57:18.089793: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Data

In [2]:
df = pd.read_csv('https://github.com/allanspadini/curso-tensorflow-proxima-palavra/raw/main/dados/train.zip', 
                 names=['ClassIndex', 'Título', 'Descrição'], header=None)

In [3]:
df.head()

,ClassIndex,Título,Descrição
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


# Preprocessing

## Features

In [5]:
df['text'] = df['Título'] + df['Descrição']

In [6]:
df.head()

,ClassIndex,Título,Descrição,text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."


In [7]:
df['ClassIndex'] -= 1

In [9]:
df['ClassIndex'].unique()

array([2, 3, 1, 0])

## Split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df['text'].values, df['ClassIndex'].values, random_state=7, test_size=0.2)

## Encoder

In [14]:
VOCAB_SIZE = 1_000

encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)

2025-02-27 10:58:33.540592: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [15]:
encoder.adapt(X_train)

In [17]:
encoder.get_vocabulary()[:5]

['', '[UNK]', np.str_('the'), np.str_('to'), np.str_('a')]

In [19]:
encoder('This is an example')

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([41, 16, 24,  1])>

# Model

In [21]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=16,
        mask_zero=True
    ),
    tf.keras.layers.GlobalAveragePooling1D(), # Dimensionality reduction
    tf.keras.layers.Dense(16, activation='relu'), # Hidden layer
    tf.keras.layers.Dense(4, activation='softmax') # Output
])

In [33]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (1, None)              │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (1, None, 16)          │        16,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (1, 16)                │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (1, 16)                │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (1, 4)                 │            68 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,340 (63.83 KB)

 Trainable params: 16,340 (63.83 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['Accuracy']
)

In [34]:
history = model.fit(x=X_train, y=y_train, epochs=10, validation_split=0.1)

Epoch 1/10
2700/2700 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - Accuracy: 0.4786 - loss: 1.3267 - val_Accuracy: 0.7533 - val_loss: 0.9573
Epoch 2/10
2700/2700 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - Accuracy: 0.7636 - loss: 0.8543 - val_Accuracy: 0.8030 - val_loss: 0.6293
Epoch 3/10
2700/2700 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - Accuracy: 0.8081 - loss: 0.6023 - val_Accuracy: 0.8236 - val_loss: 0.5273
Epoch 4/10
2700/2700 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - Accuracy: 0.8255 - loss: 0.5218 - val_Accuracy: 0.8367 - val_loss: 0.4855
Epoch 5/10
2700/2700 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - Accuracy: 0.8388 - loss: 0.4789 - val_Accuracy: 0.8430 - val_loss: 0.4633
Epoch 6/10
2700/2700 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - Accuracy: 0.8422 - loss: 0.4609 - val_Accuracy: 0.8461 - val_loss: 0.4482
Epoch 7/10
2700/2700 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - Accuracy: 0.8479 - loss: 0.4420 - val_Accuracy: 0.8477 - val_loss: 0.4376
Epoch 8/10
2700/2700 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - Accuracy: 0.8534 - loss: 0

In [35]:
model.predict(X_test[0:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


array([[2.41511315e-03, 9.97438312e-01, 2.98898049e-05, 1.16686875e-04]],
      dtype=float32)

In [36]:
model.predict(X_test[0:1]).argmax(axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


array([1])

In [37]:
y_test[0]

np.int64(1)